<a href="https://colab.research.google.com/github/juliatessler/1s2023-unicamp-dl-for-search-systems/blob/main/1-boolean-bow-tfidf-search-systems/1_Buscador_Booleano_bag_of_words_e_buscador_com_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Usar o BM25 implementado pelo pyserini para buscar queries no TREC-DL 2020
Documentação referencia: https://github.com/castorini/pyserini/blob/master/docs/experiments-msmarco-passage.md
2. Implementar um buscador booleano/bag-of-words.
3. Implementar um buscador com TF-IDF
4. Avaliar implementações 1, 2, e 3 no TREC-DL 2020 e calcular o nDCG@10
Nos itens 2 e 3:
Fazer uma implementação que suporta buscar eficientemente milhões de documentos.
Não se pode usar bibliotecas como sklearn, que já implementam o BoW e TF-IDF.

## Installing dependencies and preparing the environment

The following chunks of code are from [Pyserini development install guide](https://github.com/castorini/pyserini/blob/master/docs/installation.md#development-installation). Special thanks to Marcus Borela and Monique Louise, for sharing their code and insightful discussions on the WhatsApp group.

In [2]:
# install conda for Google Colab (thanks Monique!)
!pip install condacolab -q

import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [14]:
# install Pyserini dev dependencies
!conda install -c conda-forge maven
!pip install torch
!pip install faiss-cpu -q -c pytorch

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

# All requested packages already installed.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 47.0 MB/s eta 0:00:00
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pytorch'


In [4]:
# In order to perform some of the steps, it is needed to mount your Google Drive
# You will need to authorize 
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
main_path = '/content/drive/MyDrive/week1-notebook/'

import os

if not os.path.exists(main_path):
  os.makedirs(main_path)
else:
  print('Path already exists')

Path already exists


<!-- In order to be able do perform `git clone` from Google Colab, you should follow those steps to authorize t -->

In [10]:
!git clone https://github.com/castorini/pyserini.git --recurse-submodules

Cloning into 'pyserini'...
remote: Enumerating objects: 7217, done.
remote: Counting objects: 100% (263/263), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 7217 (delta 155), reused 195 (delta 115), pack-reused 6954
Receiving objects: 100% (7217/7217), 4.25 MiB | 13.96 MiB/s, done.
Resolving deltas: 100% (5201/5201), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 'tools'
Cloning into '/content/pyserini/tools'...
remote: Enumerating objects: 723, done.        
remote: Counting objects: 100% (480/480), done.        
remote: Compressing objects: 100% (418/418), done.        
remote: Total 723 (delta 74), reused 453 (delta 61), pack-reused 243        
Receiving objects: 100% (723/723), 57.78 MiB | 17.35 MiB/s, done.
Resolving deltas: 100% (158/158), done.
Submodule path 'tools': checked out '7b84f773225b5973b4533dfa0aa18653409a6146'


In [11]:
!cd pyserini
!cd tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
!cd tools/eval/ndeval && make && cd ../../..

/bin/bash: line 0: cd: tools/eval: No such file or directory
/bin/bash: line 0: cd: tools/eval/ndeval: No such file or directory


In [15]:
!pip install -e pyserini -q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/pyserini
  Preparing metadata (setup.py) ... done
  Attempting uninstall: pyserini
    Found existing installation: pyserini 0.20.0
    Uninstalling pyserini-0.20.0:
      Successfully uninstalled pyserini-0.20.0
  Running setup.py develop for pyserini


In [21]:
!python -m spacy download en_core_web_sm

/usr/local/lib/python3.8/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 36.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### Clone and build Anserini package
As pointed by [Pyserini development installation](https://github.com/castorini/pyserini/blob/master/docs/installation.md#development-installation), the next step is to clone and build Anserini following the [documentation steps](https://github.com/castorini/anserini#-getting-started).

In [22]:
!git clone https://github.com/castorini/anserini.git --recurse-submodules

Cloning into 'anserini'...
remote: Enumerating objects: 25522, done.
remote: Counting objects: 100% (1118/1118), done.
remote: Compressing objects: 100% (637/637), done.
remote: Total 25522 (delta 663), reused 812 (delta 420), pack-reused 24404
Receiving objects: 100% (25522/25522), 79.61 MiB | 25.51 MiB/s, done.
Resolving deltas: 100% (16395/16395), done.
Updating files: 100% (2263/2263), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 'tools'
Cloning into '/content/anserini/tools'...
remote: Enumerating objects: 723, done.        
remote: Counting objects: 100% (480/480), done.        
remote: Compressing objects: 100% (418/418), done.        
remote: Total 723 (delta 74), reused 453 (delta 61), pack-reused 243        
Receiving objects: 100% (723/723), 57.78 MiB | 16.78 MiB/s, done.
Resolving deltas: 100% (158/158), done.
Submodule path 'tools': checked out '95fbaf2af75e2b59304ac5702d5479d50f3bd9ef'


In [24]:
# Thanks Monique, once again, for figuring that out
!mvn -f anserini/pom.xml clean package appassembler:assemble

[INFO] Scanning for projects...
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.8.1/maven-compiler-plugin-3.8.1.pom (12 kB at 22 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/33/maven-plugins-33.pom (11 kB at 261 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/33/maven-parent-33.pom (44 kB at 833 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/21/apache-21.pom (17 kB at 658 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.8.1/maven-compiler-plugin-3.8.1.jar (62 kB at 1.5 MB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/mojo/appassembler-maven-plugin/2.1.0/appassembler-maven-plugin-2.1.0.pom (19 kB at 547 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/code

In [26]:
!cd tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
!cd tools/eval/ndeval && make && cd ../../..


/bin/bash: line 0: cd: tools/eval: No such file or directory
/bin/bash: line 0: cd: tools/eval/ndeval: No such file or directory


## Get dataset
Following the instructions from the [reference docs](https://github.com/castorini/pyserini/blob/master/docs/experiments-msmarco-passage.md) provided to solve the problem, let's download the data.

In [16]:
# creates a new working directory
!mkdir collections/msmarco-passage

# downloads the data to the newly created directory 
!wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz -P collections/msmarco-passage

# extracts data
!tar xvfz collections/msmarco-passage/collectionandqueries.tar.gz -C collections/msmarco-passage

mkdir: cannot create directory ‘collections/msmarco-passage’: File exists
--2023-03-06 01:48:39--  https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1057717952 (1009M) [application/gzip]
Saving to: ‘collections/msmarco-passage/collectionandqueries.tar.gz.1’

collectionandquerie 100%[===================>]   1009M  10.4MB/s    in 83s     

2023-03-06 01:50:03 (12.1 MB/s) - ‘collections/msmarco-passage/collectionandqueries.tar.gz.1’ saved [1057717952/1057717952]

collection.tsv
qrels.dev.small.tsv
qrels.train.tsv
queries.dev.small.tsv
queries.dev.tsv
queries.eval.small.tsv
queries.eval.tsv
queries.train.tsv


Next, the data needs to be converted to JSONL files, which can be easily achieved through the provided functions.

In [18]:
!python pyserini/tools/scripts/msmarco/convert_collection_to_jsonl.py \
 --collection-path collections/msmarco-passage/collection.tsv \
 --output-folder collections/msmarco-passage/collection_jsonl

Converting collection...
Converted 0 docs, writing into file 1
Converted 100,000 docs, writing into file 1
Converted 200,000 docs, writing into file 1
Converted 300,000 docs, writing into file 1
Converted 400,000 docs, writing into file 1
Converted 500,000 docs, writing into file 1
Converted 600,000 docs, writing into file 1
Converted 700,000 docs, writing into file 1
Converted 800,000 docs, writing into file 1
Converted 900,000 docs, writing into file 1
Converted 1,000,000 docs, writing into file 2
Converted 1,100,000 docs, writing into file 2
Converted 1,200,000 docs, writing into file 2
Converted 1,300,000 docs, writing into file 2
Converted 1,400,000 docs, writing into file 2
Converted 1,500,000 docs, writing into file 2
Converted 1,600,000 docs, writing into file 2
Converted 1,700,000 docs, writing into file 2
Converted 1,800,000 docs, writing into file 2
Converted 1,900,000 docs, writing into file 2
Converted 2,000,000 docs, writing into file 3
Converted 2,100,000 docs, writing i

In [20]:
!python -m pyserini/tools/scripts/msmarco/pyserini.index.lucene \
  --collection JsonCollection \
  --input collections/msmarco-passage/collection_jsonl \
  --index indexes/lucene-index-msmarco-passage \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/runpy.py", line 185, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/local/lib/python3.8/runpy.py", line 111, in _get_module_details
    __import__(pkg_name)
  File "/content/pyserini/pyserini/index/__init__.py", line 21, in <module>
    from .lucene._base import Document, Generator, IndexTerm, Posting, IndexReader
  File "/content/pyserini/pyserini/index/lucene/__init__.py", line 17, in <module>
    from ._base import Document, Generator, IndexTerm, Posting, IndexReader
  File "/content/pyserini/pyserini/index/lucene/_base.py", line 30, in <module>
    from pyserini.analysis import get_lucene_analyzer, JAnalyzer, JAnalyzerUtils
  File "/content/pyserini/pyserini/analysis/__init__.py", line 17, in <module>
    from ._base import get_lucene_analyzer, Analyzer, JAnalyzer, JAnalyzerUtils, JDefaultEnglishAnalyzer, JWhiteSpaceAnalyzer
  File "/content/pyserini/pyserin

In [7]:
!head tools/topics-and-qrels/topics.msmarco-passage.dev-subset.txt

head: cannot open 'tools/topics-and-qrels/topics.msmarco-passage.dev-subset.txt' for reading: No such file or directory


In [10]:
!python -m pyserini.search.lucene \
  --index indexes/lucene-index-msmarco-passage \
  --topics msmarco-passage-dev-subset \
  --output runs/run.msmarco-passage.bm25tuned.txt \
  --output-format msmarco \
  --hits 1000 \
  --bm25 --k1 0.82 --b 0.68

2023-03-04 10:57:57.276626: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 10:58:00.876731: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-04 10:58:00.877274: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-04 10:58:00.877296: W tensorfl